# 환경설정

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [2]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import cv2
import pandas as pd

from pathlib import Path

import torchvision
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F
import torchvision.transforms.v2 as v2
from torchvision.transforms import v2 as T
from PIL import Image, ImageEnhance, ImageFilter
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# YOLO v5 설정 및 확인
from ultralytics import YOLO
import os
import shutil

In [ ]:
print("PyTorch:", torch.__version__)
print("MPS available:", torch.backends.mps.is_available())
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

os.makedirs("pth", exist_ok=True)

## 한글세팅

※ macOS에서 구동하는 한글 세팅임.

In [ ]:
# macOS용 한글 폰트 설정

# macOS에서 사용 가능한 한글 폰트들
macos_korean_fonts = [
    '/System/Library/Fonts/AppleGothic.ttf',
    '/System/Library/Fonts/AppleSDGothicNeo.ttc',
    '/Library/Fonts/NanumGothic.ttf',
    '/Library/Fonts/NanumBarunGothic.ttf',
    '/System/Library/Fonts/PingFang.ttc'
]

# 사용 가능한 폰트 찾기
available_font = None
for font_path in macos_korean_fonts:
    if os.path.exists(font_path):
        available_font = font_path
        print(f"사용 가능한 폰트 발견: {font_path}")
        break

if available_font:
    # 폰트 설정
    font_prop = fm.FontProperties(fname=available_font)
    plt.rcParams['font.family'] = font_prop.get_name()
    plt.rcParams['axes.unicode_minus'] = False
    print(f"폰트 설정 완료: {font_prop.get_name()}")
else:
    # 기본 폰트로 설정
    plt.rcParams['font.family'] = 'AppleGothic'
    plt.rcParams['axes.unicode_minus'] = False
    print("기본 AppleGothic 폰트 사용")

## 함수화

In [5]:
# 모델 저장 함수
def save_model(model, save_path, model_name="drug_detection_model"):
    """훈련된 모델을 저장하는 함수"""
    import os
    from datetime import datetime
    
    # 저장 디렉토리 생성
    os.makedirs(save_path, exist_ok=True)
    
    # 현재 시간을 포함한 파일명 생성
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_filename = f"{model_name}_{timestamp}.pt"
    model_path = os.path.join(save_path, model_filename)
    
    # 모델 저장
    model.save(model_path)
    
    print(f"모델이 저장되었습니다: {model_path}")
    return model_path
    
save_dir = "../models"


## 데이터 불러오기

In [ ]:
# 경로 설정
RAW_DATA_PATH = '../data/raw_data'
TRAIN_IMAGES_PATH = os.path.join(RAW_DATA_PATH, 'train_images')
TRAIN_ANNOTATIONS_PATH = os.path.join(RAW_DATA_PATH, 'train_annotations')
TEST_IMAGES_PATH = os.path.join(RAW_DATA_PATH, 'test_images')

print("데이터 경로 확인:")
print(f"Train Images: {TRAIN_IMAGES_PATH}")
print(f"Train Annotations: {TRAIN_ANNOTATIONS_PATH}")
print(f"Test Images: {TEST_IMAGES_PATH}")


# COCO 데이터 파싱

In [ ]:
# COCO 형식 어노테이션 파일들을 로드하고 통합
def load_coco_annotations(annotations_path):
    all_annotations = []
    all_images = []
    all_categories = []
    
    # 모든 JSON 파일 찾기
    json_files = []
    for root, dirs, files in os.walk(annotations_path):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(root, file))
    
    print(f"어노테이션 파일 : 총 {len(json_files)}개")
    
    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                
            # 이미지 정보 추가
            if 'images' in data:
                all_images.extend(data['images'])
            
            # 어노테이션 정보 추가
            if 'annotations' in data:
                all_annotations.extend(data['annotations'])
            
            # 카테고리 정보 추가
            if 'categories' in data:
                all_categories.extend(data['categories'])
                
        except Exception as e:
            print(f"Error loading {json_file}: {e}")
    
    return all_images, all_annotations, all_categories

# 데이터 로드
images, annotations, categories = load_coco_annotations(TRAIN_ANNOTATIONS_PATH)

print(f"총 이미지 수: {len(images)}")
print(f"총 어노테이션 수: {len(annotations)}")
print(f"총 카테고리 수: {len(categories)}")

# 데이터프레임으로 변환
df_images = pd.DataFrame(images)
df_annotations = pd.DataFrame(annotations)
df_categories = pd.DataFrame(categories)

print("\n이미지 데이터 샘플:")
print(df_images.head())
print("\n어노테이션 데이터 샘플:")
print(df_annotations.head())
print("\n카테고리 데이터 샘플:")
print(df_categories.head())

# 데이터 분석

In [ ]:
# 이미지 크기 분석
print("이미지 크기 통계:")
print(df_images[['width', 'height']].describe())

# 이미지 크기 분포 시각화
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df_images['width'], bins=30, alpha=0.7, color='skyblue')
plt.title('Image Width Distribution')
plt.xlabel('Width (pixels)')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
plt.hist(df_images['height'], bins=30, alpha=0.7, color='lightcoral')
plt.title('Image Height Distribution')
plt.xlabel('Height (pixels)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# 카테고리별 분포
category_counts = df_categories['name'].value_counts()
print(f"\n총 약 종류 수: {len(category_counts)}")

plt.figure(figsize=(15, 8))
category_counts.head(20).plot(kind='bar')
plt.title('Top 20 Drug Categories')
plt.xlabel('Drug Name')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
category_counts.head(20)

In [ ]:
# 바운딩 박스 크기 분석
bbox_areas = df_annotations['area'].values
bbox_widths = [bbox[2] for bbox in df_annotations['bbox']]
bbox_heights = [bbox[3] for bbox in df_annotations['bbox']]

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(bbox_areas, bins=50, alpha=0.7, color='lightgreen')
plt.title('Bounding Box Area Distribution')
plt.xlabel('Area (pixels²)')
plt.ylabel('Frequency')

plt.subplot(1, 3, 2)
plt.hist(bbox_widths, bins=50, alpha=0.7, color='orange')
plt.title('Bounding Box Width Distribution')
plt.xlabel('Width (pixels)')
plt.ylabel('Frequency')

plt.subplot(1, 3, 3)
plt.hist(bbox_heights, bins=50, alpha=0.7, color='purple')
plt.title('Bounding Box Height Distribution')
plt.xlabel('Height (pixels)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## 데이터 분석
- 이미지 특성
    - 이미지 크기 : 976 x 1280 픽셀 (모두 같음)
    - 총 이미지 수 : 2,332개 (train : 1489개, test : 843개)

- 약 종류 분포
    - 총 약 종류 : 73개
    - Top rank 1 : 가네신에프정(은행엽엑스)(수출용) [514개]
    - Top rank 2 : 일양하이트린정 2mg [240개]
    - Top rank 3 : 보령부스파정 5mg [180개]

    - 클래스 불균형 문제: 상위 약들이 압도적으로 많음

- 바운딩 박스 특성
    - 면적 : 50,000 픽셀^2 이하
    - 너비 : 주로 200~225 픽셀
    - 높이 : 200~250 픽셀, 475~500 픽셀 등


## 이미지 전처리

In [ ]:
# 이미지 전처리
def preprocess_image(image_path, target_size=(640, 640)):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"이미지를 로드 불가 : {image_path}")
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # BGR to RGB 변환
    
    original_height, original_width = image.shape[:2] # 원본 크기 저장
    
    resized_image = cv2.resize(image, target_size) # resize
    
    normalized_image = resized_image.astype(np.float32) / 255.0 # 정규화 (0~1)
    
    return {
        'image': normalized_image,
        'original_size': (original_width, original_height),
        'target_size': target_size
    }

# 바운딩 박스 조정
def adjust_bbox(bbox, original_size, target_size):
    x, y, w, h = bbox
    orig_w, orig_h = original_size
    target_w, target_h = target_size
    
    # 스케일 계산
    scale_x = target_w / orig_w
    scale_y = target_h / orig_h
    
    # 좌표 변환
    new_x = x * scale_x
    new_y = y * scale_y
    new_w = w * scale_x
    new_h = h * scale_y
    
    return [new_x, new_y, new_w, new_h]

# 샘플 이미지로 전처리 테스트
sample_image_path = os.path.join(TRAIN_IMAGES_PATH, df_images.iloc[0]['file_name'])
sample_annotation = df_annotations.iloc[0]

try:
    processed = preprocess_image(sample_image_path)
    adjusted_bbox = adjust_bbox(
        sample_annotation['bbox'], 
        processed['original_size'], 
        processed['target_size']
    )
    
    print("전처리 결과:")
    print(f"원본 크기: {processed['original_size']}")
    print(f"변환 크기: {processed['target_size']}")
    print(f"원본 바운딩 박스: {sample_annotation['bbox']}")
    print(f"조정된 바운딩 박스: {adjusted_bbox}")
    
    #########################################################
    # 시각화
    plt.figure(figsize=(12, 6))
    
    # 원본 이미지
    original_img = cv2.imread(sample_image_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    plt.subplot(1, 2, 1)
    plt.imshow(original_img)

    # 원본 바운딩 박스 시각화
    x, y, w, h = sample_annotation['bbox']
    rect = plt.Rectangle((x, y), w, h, linewidth=2, edgecolor='red', facecolor='none')
    plt.gca().add_patch(rect)
    plt.title('Original Image with BBox')
    plt.axis('off')
    
    #########################################################
    # 전처리된 이미지
    plt.subplot(1, 2, 2)
    plt.imshow(processed['image'])

    # 조정된 바운딩 박스 시각화
    x, y, w, h = adjusted_bbox
    rect = plt.Rectangle((x, y), w, h, linewidth=2, edgecolor='red', facecolor='none')
    plt.gca().add_patch(rect)
    plt.title('Preprocessed Image with Adjusted BBox')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"전처리 테스트 중 오류: {e}")

## 데이터 증강

In [13]:
class DrugDetectionTransforms:

    def __init__(self, is_training=True, target_size=(640, 640)):
        self.is_training = is_training
        self.target_size = target_size
        
        if is_training:
            self.transform = v2.Compose([
                v2.Resize(target_size),
                #v2.RandomHorizontalFlip(p=0.5),
                #v2.RandomVerticalFlip(p=0.3),
                #v2.RandomRotation(degrees=10),
                #v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
                #v2.RandomGrayscale(p=0.1),
                v2.ToTensor(),
                v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        else:
            self.transform = v2.Compose([
                v2.Resize(target_size),
                v2.ToTensor(),
                v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
    
    def __call__(self, image, bboxes=None, labels=None):
        # PIL Image로 변환
        if isinstance(image, np.ndarray):
            image = Image.fromarray(image)
        
        # 이미지 변환
        transformed_image = self.transform(image)
        
        # 바운딩 박스 조정
        if bboxes is not None and self.is_training:
            # 랜덤 변환에 따른 바운딩 박스 조정
            adjusted_bboxes = self._adjust_bboxes(bboxes, image.size, self.target_size)
            return transformed_image, adjusted_bboxes, labels
        
        return transformed_image, bboxes, labels
    
    # 바운딩 박스 조정
    def _adjust_bboxes(self, bboxes, original_size, target_size):
        orig_w, orig_h = original_size
        target_w, target_h = target_size
        
        scale_x = target_w / orig_w
        scale_y = target_h / orig_h
        
        adjusted_bboxes = []
        for bbox in bboxes:
            x, y, w, h = bbox
            new_x = x * scale_x
            new_y = y * scale_y
            new_w = w * scale_x
            new_h = h * scale_y
            adjusted_bboxes.append([new_x, new_y, new_w, new_h])
        
        return adjusted_bboxes

## 데이터 셋

In [14]:
class DrugDetectionDataset(Dataset):
    
    def __init__(self, images_df, annotations_df, categories_df, 
                 images_path, transform=None, target_size=(640, 640)):
        self.images_df = images_df
        self.annotations_df = annotations_df
        self.categories_df = categories_df
        self.images_path = images_path
        self.transform = transform
        self.target_size = target_size
        
        # 카테고리 ID 매핑 생성
        self.category_to_id = {cat['name']: cat['id'] for _, cat in categories_df.iterrows()}
        self.id_to_category = {cat['id']: cat['name'] for _, cat in categories_df.iterrows()}
        
        # 이미지 ID별 어노테이션 그룹화
        self.image_annotations = {}
        for _, ann in annotations_df.iterrows():
            img_id = ann['image_id']
            if img_id not in self.image_annotations:
                self.image_annotations[img_id] = []
            self.image_annotations[img_id].append(ann)
    
    def __len__(self):
        return len(self.images_df)
    
    def __getitem__(self, idx):
        # 이미지 정보 가져오기
        img_info = self.images_df.iloc[idx]
        img_path = os.path.join(self.images_path, img_info['file_name'])
        
        # 이미지 로드 (PIL 사용)
        image = Image.open(img_path).convert('RGB')
        
        # 어노테이션 가져오기
        img_id = img_info['id']
        annotations = self.image_annotations.get(img_id, [])
        
        # 바운딩 박스와 라벨 준비
        bboxes = []
        labels = []
        
        for ann in annotations:
            bbox = ann['bbox']  # [x, y, w, h]
            category_id = ann['category_id']
            bboxes.append(bbox)
            labels.append(category_id)
        
        # 데이터 증강 적용
        if self.transform:
            image, bboxes, labels = self.transform(image, bboxes, labels)
        
        return {
            'image': image,
            'bboxes': torch.tensor(bboxes, dtype=torch.float32) if bboxes else torch.empty((0, 4)),
            'labels': torch.tensor(labels, dtype=torch.long) if labels else torch.empty((0,), dtype=torch.long),
            'image_id': img_id,
            'original_size': (img_info['width'], img_info['height'])
        }

In [ ]:
# 데이터 증강 파이프라인 생성
train_transform = DrugDetectionTransforms(is_training=True, target_size=(640, 640))
val_transform = DrugDetectionTransforms(is_training=False, target_size=(640, 640))

# 데이터셋 생성
train_dataset = DrugDetectionDataset(
    df_images, df_annotations, df_categories, 
    TRAIN_IMAGES_PATH, 
    transform=train_transform
)

# 데이터로더 생성
train_loader = DataLoader(
    train_dataset, 
    batch_size=8, 
    shuffle=True, 
    num_workers=2
)

print(f"데이터셋 크기: {len(train_dataset)}")
print(f"데이터로더 배치 수: {len(train_loader)}")

------

# 모델

## YOLO
 - https://docs.ultralytics.com/models/

### YOLO v5

#### YOLO v5 설정

In [16]:
# YOLO v5 모델 로드 테스트
model = YOLO('yolov8n.pt')
print("YOLO v5 모델 로드 성공!")

# 현재 디렉토리 구조 확인
print(f"현재 작업 디렉토리: {os.getcwd()}")

YOLO v5 모델 로드 성공!
현재 작업 디렉토리: /Users/leeyoungho/develop/ai_study/project/sprint_ai03_1/notebooks


#### YOLO 데이터셋 변환

In [17]:
# COCO 형식을 YOLO 형식으로 변환
def convert_to_yolo_format(images_df, annotations_df, categories_df, 
                          images_path, output_path):   
    # 출력 디렉토리 생성
    os.makedirs(output_path, exist_ok=True)
    os.makedirs(os.path.join(output_path, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_path, 'labels'), exist_ok=True)
    
    # 고유한 카테고리만 추출하고 0부터 시작하는 인덱스 매핑
    unique_categories = categories_df['name'].unique()
    category_mapping = {}
    
    # 원본 카테고리 ID를 0부터 시작하는 인덱스로 매핑
    for idx, cat_name in enumerate(unique_categories):
        # 해당 이름을 가진 카테고리의 원본 ID 찾기
        original_id = categories_df[categories_df['name'] == cat_name]['id'].iloc[0]
        category_mapping[original_id] = idx
    
    print(f"총 {len(category_mapping)}개 카테고리 매핑 완료")
    print(f"매핑 예시: {list(category_mapping.items())[:5]}")
    
    # 이미지 ID별 어노테이션 그룹화
    image_annotations = {}
    for _, ann in annotations_df.iterrows():
        img_id = ann['image_id']
        if img_id not in image_annotations:
            image_annotations[img_id] = []
        image_annotations[img_id].append(ann)
    
    # 각 이미지에 대해 YOLO 형식으로 변환
    for idx, (_, img_info) in enumerate(images_df.iterrows()):
        img_id = img_info['id']
        img_filename = img_info['file_name']
        
        # 이미지 복사
        src_img_path = os.path.join(images_path, img_filename)
        dst_img_path = os.path.join(output_path, 'images', img_filename)
        shutil.copy2(src_img_path, dst_img_path)
        
        # 라벨 파일 생성
        label_filename = img_filename.replace('.png', '.txt')
        label_path = os.path.join(output_path, 'labels', label_filename)
        
        annotations = image_annotations.get(img_id, [])
        
        with open(label_path, 'w') as f:
            for ann in annotations:
                # 바운딩 박스 좌표 (COCO: [x, y, w, h] -> YOLO: [x_center, y_center, w, h])
                bbox = ann['bbox']
                x, y, w, h = bbox
                
                # 이미지 크기
                img_width = img_info['width']
                img_height = img_info['height']
                
                # YOLO 형식으로 변환 (정규화된 좌표)
                x_center = (x + w/2) / img_width
                y_center = (y + h/2) / img_height
                w_norm = w / img_width
                h_norm = h / img_height
                
                # 카테고리 ID (YOLO는 0부터 시작)
                category_id = category_mapping[ann['category_id']]
                
                # YOLO 형식: class_id x_center y_center width height
                f.write(f"{category_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")
        
        if (idx + 1) % 100 == 0:
            print(f"진행률: {idx + 1}/{len(images_df)}")
    
    # classes.txt 파일 생성
    classes_path = os.path.join(output_path, 'classes.txt')
    with open(classes_path, 'w', encoding='utf-8') as f:
        for _, cat in categories_df.iterrows():
            f.write(f"{cat['name']}\n")
    
    print(f"YOLO 형식 변환 완료! 출력 경로: {output_path}")
    return output_path

In [ ]:
# 데이터셋 분할 (훈련/검증)

if os.path.exists('../data/data_lyh/yolo_dataset'):
    shutil.rmtree('../data/data_lyh/yolo_dataset')
    
train_images, val_images = train_test_split(
    df_images, test_size=0.2, random_state=42
)

print(f"훈련 데이터: {len(train_images)}개")
print(f"검증 데이터: {len(val_images)}개")

# YOLO 형식으로 변환
yolo_dataset_path = '../data/data_lyh/yolo_dataset'

# 훈련 데이터 변환
train_path = os.path.join(yolo_dataset_path, 'train')
convert_to_yolo_format(
    train_images, df_annotations, df_categories, 
    TRAIN_IMAGES_PATH, train_path
)

# 검증 데이터 변환
val_path = os.path.join(yolo_dataset_path, 'val')
convert_to_yolo_format(
    val_images, df_annotations, df_categories, 
    TRAIN_IMAGES_PATH, val_path
)

print("데이터셋 변환 완료!")

In [ ]:
# 카테고리 수 변화 원인 확인
print("이전 고유 카테고리 수:", len(df_categories['name'].unique()))

# 중복 제거 확인
unique_categories = df_categories['name'].unique()
print("실제 고유 카테고리:", len(unique_categories))

In [ ]:
# 라벨 파일 내용 확인
import os
label_files = os.listdir('../data/data_lyh/yolo_dataset/train/labels')
sample_label_path = os.path.join('../data/data_lyh/yolo_dataset/train/labels', label_files[0])

with open(sample_label_path, 'r') as f:
    print("샘플 라벨 파일 내용:")
    print(f.read())

In [ ]:
# YAML 파일 내용 확인
"""
with open('../data/data_lyh/yolo_dataset/dataset.yaml', 'r') as f:
    print("YAML 파일 내용:")
    print(f.read())
"""

In [22]:
# 현재 매핑 상태 확인
print("원본 카테고리 ID들:")
print(df_categories['id'].unique()[:10])  # 처음 10개 확인

#print("\n매핑된 카테고리 ID들:")
#print(list(category_mapping.items())[:10])  # 처음 10개 확인

원본 카테고리 ID들:
[27776 25366  3482 35205 36636 27732 28762 27925  1899 24849]


#### YOLO 설정 파일 생성

In [23]:
# 카테고리별 분포
category_counts = df_categories['name'].value_counts()
print(f"\n총 약 종류 수: {len(category_counts)}")


총 약 종류 수: 74


In [24]:
# 중복된 카테고리 제거하고 고유한 약 이름만 추출
unique_categories = df_categories['name'].unique()
unique_categories_list = unique_categories.tolist()

print(len(unique_categories_list))
print(unique_categories_list)

74
['카나브정 60mg', '자누메트정 50/850mg', '기넥신에프정(은행엽엑스)(수출용)', '아토젯정 10/40mg', '로수젯정10/5밀리그램', '트윈스타정 40/5mg', '트라젠타정(리나글립틴)', '울트라셋이알서방정', '보령부스파정 5mg', '놀텍정 10mg', '동아가바펜틴정 800mg', '세비카정 10/40mg', '자누비아정 50mg', '플라빅스정 75mg', '크레스토정 20mg', '가바토파정 100mg', '라비에트정 20mg', '레일라정', '일양하이트린정 2mg', '리피토정 20mg', '글리틴정(콜린알포세레이트)', '비모보정 500/20mg', '카발린캡슐 25mg', '비타비백정 100mg/병', '써스펜8시간이알서방정 650mg', '큐시드정 31.5mg/PTP', '뮤테란캡슐 100mg', '조인스정 200mg', '자이프렉사정 2.5mg', '무코스타정(레바미피드)(비매품)', '자누메트엑스알서방정 100/1000mg', '리바로정 4mg', '노바스크정 5mg', '에빅사정(메만틴염산염)(비매품)', '마도파정', '오마코연질캡슐(오메가-3-산에틸에스테르90)', '스토가정 10mg', '엑스포지정 5/160mg', '알드린정', '리렉스펜정 300mg/PTP', '제미메트서방정 50/1000mg', '타이레놀이알서방정(아세트아미노펜)(수출용)', '삐콤씨에프정 618.6mg/병', '리피로우정 20mg', '뉴로메드정(옥시라세탐)', '리리카캡슐 150mg', '낙소졸정 500/20mg', '란스톤엘에프디티정 30mg', '트루비타정 60mg/병', '맥시부펜이알정 300mg', '트라젠타듀오정 2.5/850mg', '아빌리파이정 10mg', '콜리네이트연질캡슐 400mg', '종근당글리아티린연질캡슐(콜린알포세레이트)\xa0', '아토르바정 10mg', '쎄로켈정 100mg', '신바로정', '다보타민큐정 10mg/병', '글리아타민연질캡슐', '아모잘탄정 5/100mg', '케이캡정 50mg', '삼

In [ ]:
# YOLO 설정 파일 생성
yaml_content = f"""
# YOLO v5 설정 파일
path: {os.path.abspath(yolo_dataset_path)}  # 데이터셋 루트 경로
train: train/images  # 훈련 이미지 경로
val: val/images      # 검증 이미지 경로

# 클래스 수 (고유한 약 종류 수)
nc: {len(unique_categories)}  # 클래스 개수

# 클래스 이름
names: {unique_categories_list}
"""

# 설정 파일 저장
yaml_path = os.path.join(yolo_dataset_path, 'dataset.yaml')
with open(yaml_path, 'w', encoding='utf-8') as f:
    f.write(yaml_content)

print(f"YOLO 설정 파일 생성 완료: {yaml_path}")
print(f"총 클래스 수: {len(unique_categories)}")
print(f"고유한 약 종류: {len(unique_categories)}개")

# YAML 파일 내용 확인
print("\nYAML 파일 내용:")
with open(yaml_path, 'r', encoding='utf-8') as f:
    print(f.read())

In [ ]:
# 라벨 파일 내용 확인
label_path = '../data/data_lyh/yolo_dataset/train/labels/K-001900-010224-016551-031705_0_2_0_2_70_000_200.txt'
with open(label_path, 'r') as f:
    print(f.read())

In [ ]:
# YAML 파일 내용 확인
with open('../data/data_lyh/yolo_dataset/dataset.yaml', 'r') as f:
    print(f.read())

#### YOLO 모델 학습

In [ ]:
# YOLO 모델 학습
def train_yolo_model():
    """YOLO v5 모델 학습"""
    
    # 모델 초기화
    model = YOLO('yolov8n.pt')  # 사전 훈련된 모델 로드
    
    # 학습 설정
    results = model.train(
        data=yaml_path,           # 데이터셋 설정 파일
        epochs=100,               # 에포크 수
        imgsz=640,               # 이미지 크기
        batch=8,                 # 배치 크기
        device=device,           # 디바이스 (MPS/CPU)
        patience=10,             # Early stopping patience
        save=True,               # 모델 저장
        project='drug_detection', # 프로젝트 이름
        name='yolov8x_drug_model', # 실험 이름
        exist_ok=True,           # 기존 실험 덮어쓰기
        pretrained=True,         # 사전 훈련된 가중치 사용
        optimizer='Adam',        # 옵티마이저
        lr0=0.001,              # 초기 학습률
        weight_decay=0.0005,     # 가중치 감쇠
        warmup_epochs=3,         # 워밍업 에포크
        cos_lr=True,            # 코사인 학습률 스케줄링
        label_smoothing=0.1,     # 라벨 스무딩
        mixup=0.1,              # Mixup 증강
        mosaic=1.0,             # Mosaic 증강
        degrees=10.0,           # 회전 증강
        translate=0.1,          # 이동 증강
        scale=0.5,              # 스케일 증강
        shear=2.0,              # 전단 증강
        perspective=0.0,         # 원근 증강
        flipud=0.0,             # 상하 뒤집기
        fliplr=0.5,             # 좌우 뒤집기
        hsv_h=0.015,            # HSV 색조 증강
        hsv_s=0.7,              # HSV 채도 증강
        hsv_v=0.4,              # HSV 명도 증강
        max_det=50,            # 최대 검출 개수
    )
    
    return results, model

# 학습 시작
print("YOLO 모델 학습 시작...")
results, trained_model = train_yolo_model()
print("학습 완료!")

saved_model_path = save_model(trained_model, save_dir)

# 학습 결과 시각화
results.plot()  # 학습 곡선 플롯

### 모델 불러오기

In [ ]:
# 모델 불러오기 함수
def load_model(model_path):    
    try:
        # 모델 로드
        model = YOLO(model_path)
        print(f"모델이 성공적으로 로드되었습니다: {model_path}")
        return model
    except Exception as e:
        print(f"모델 로드 중 오류 발생: {e}")
        return None

# 모델 불러오기 실행
loaded_model = load_model(saved_model_path)

### 모델 성능 평가

In [ ]:
# 모델 성능 평가 함수
def evaluate_model(model, data_path):
    
    # 검증 데이터셋에서 평가
    results = model.val(data=data_path)
    
    # 결과 출력
    print("=== 모델 성능 평가 결과 ===")
    print(f"mAP50: {results.box.map50:.3f}")
    print(f"mAP50-95: {results.box.map:.3f}")
    print(f"Precision: {results.box.mp:.3f}")
    print(f"Recall: {results.box.mr:.3f}")
    
    return results

# 모델 평가 실행
if loaded_model:
    evaluation_results = evaluate_model(loaded_model, yaml_path)

### 모델 테스트

In [ ]:
# 모델 추론 테스트 함수
def test_inference(model, image_path):
    
    # 이미지 로드
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # 추론 실행
    results = model(image_path)
    
    # 결과 시각화
    for result in results:
        boxes = result.boxes
        if boxes is not None:
            # 바운딩 박스 그리기
            for box in boxes:
                # 좌표 추출
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                conf = box.conf[0].cpu().numpy()
                cls = int(box.cls[0].cpu().numpy())
                
                # 박스 그리기
                cv2.rectangle(image_rgb, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
                
                # 라벨 추가
                label = f"Class {cls}: {conf:.2f}"
                cv2.putText(image_rgb, label, (int(x1), int(y1)-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    # 결과 이미지 표시
    plt.figure(figsize=(12, 8))
    plt.imshow(image_rgb)
    plt.title("Drug Detection Results")
    plt.axis('off')
    plt.show()
    
    return results

In [ ]:
# 테스트 실행 (샘플 이미지 경로 지정 필요)
sample_image_path = "../data/data_lyh/yolo_dataset/val/images/sample_image.jpg"
if loaded_model and os.path.exists(sample_image_path):
    inference_results = test_inference(loaded_model, sample_image_path)